# File Setup

In [1]:
#dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as sts
import random as random
import requests
import sys
import gmaps
from pprint import pprint

#plot style
plt.style.use('dark_background')

#api-key in different folder, so need to adjust filepath
sys.path.append("/Users/Chrismus Time/Desktop/upenn-datacamp-2021/homework-repo/python-api-challenge")

#import keys
from config import gkey
from config import api_key

#configure key
gmaps.configure(api_key = gkey)

#github commands
# jupyter nbextension enable --py gmaps
# jupyter nbextension enable --py widgetsnbextension

# Variable Setup

In [2]:
#import data from created city csv
city_df = pd.read_csv("../WeatherPy/dataframe.csv")

#clean up data
clean_df = city_df.dropna(how = 'any').reset_index().drop(['index', 'Unnamed: 0'], axis = 1)
clean_df.head()

,City,Country,Latitude,Longitude,Temp (F),Humidity (%),Clouds (%),Wind Speed (mph)
0,albany,au,42.652579,-73.756232,42.66,85.0,2.0,3.00
1,victoria,sc,36.994874,-78.227220,77.86,77.0,74.0,3.00
2,masuguru,tz,-11.349025,38.429997,95.56,25.0,8.0,5.70
3,hobart,au,41.532259,-87.255035,41.61,74.0,40.0,1.99
4,lebu,cl,-37.608048,-73.651491,57.15,72.0,100.0,15.90


### Adding the max/min temp to dataframe
 - misread the directions. thought we had to go back to openweatherapi and fetch the max min temps, but its just asking for a cap on the temp
 - code below works, just not what instructions asked for

In [3]:
# #creating lists to be used in for loop
# max_temp, min_temp = [], [] #creating storage variables dict
# values_list = [max_temp, min_temp] #putting these in a meta-list for looping
# function_list = ["city_response['main']['temp_max']", "city_response['main']['temp_min']"]
# city_names = clean_df['City'].tolist() #extracting all city names

# #defining varibales to be used in the for loop
# units = 'imperial' #defining units as F
# base_url = f"http://api.openweathermap.org/data/2.5/weather?appid={api_key}&units={units}&q=" #defining url

In [4]:
# #populating lists with relevant values
# counter = 0
# for city in city_names:
#     city_response = requests.get(f"{base_url}{city}").json()
#     for i in range(len(values_list)):
#         #because the citynames were pre-vetted from part 1, don't need a try/except to catch NaN, since none of the cities will have NaNs
#         values_list[i].append(eval(function_list[i]))
#     counter += 1
#     print(f"{city} #{counter} of {len(city_names)}") #adding print to track progress

In [5]:
# #adding to df
# clean_df['Max Temp (F)'] = max_temp
# clean_df['Min Temp (F)'] = min_temp
# clean_df.head()

# Plotting Humidity Heatmap

In [6]:
#figure settings
fig_settings = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#create fig
fig = gmaps.figure(layout = fig_settings)

#generating heatmap variables
coordinates = list(zip(clean_df['Latitude'].tolist(), clean_df['Longitude'].tolist()))
humidity_values = clean_df['Humidity (%)']
max_intensity = 100
point_radius = 7

#creating marker layer
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity_values, dissipating = True, max_intensity = max_intensity, point_radius = point_radius)

#testing the humidity to see why so much red
print(f"Humidity: mean {round(clean_df['Humidity (%)'].mean(),2)}, median {clean_df['Humidity (%)'].median()}, mode {clean_df['Humidity (%)'].mode()}.")

#adding markers to map
fig.add_layer(heat_layer)
fig

Humidity: mean 70.27, median 74.0, mode 0    74.0
dtype: float64.


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

# Plotting Ideal Heatmap

### Narrowing down ideal conditions

In [7]:
#using custom qualifications for personal ideal
ideal_df = clean_df.loc[(clean_df['Temp (F)'] < 77) &
                        (clean_df['Temp (F)'] > 60) &
                        (clean_df['Humidity (%)'] < 50) &
                        (clean_df['Wind Speed (mph)'] < 10) &
                        (clean_df['Clouds (%)'] > 10)]
print(f"There are only {len(ideal_df['City'])} ideal cities out of {len(clean_df['City'])}.")
ideal_df = ideal_df.reset_index().drop('index', axis = 1)
ideal_df.head()

There are only 4 ideal cities out of 460.


,City,Country,Latitude,Longitude,Temp (F),Humidity (%),Clouds (%),Wind Speed (mph)
0,ginir,et,7.138672,40.711564,69.04,37.0,85.0,7.92
1,emerald,au,37.088557,-95.580680,76.98,33.0,92.0,3.49
2,armilla,es,37.151231,-3.616715,68.29,40.0,20.0,3.00
3,rawson,ar,-43.299858,-65.099487,74.07,31.0,97.0,8.01


### Fetching data for closest hotels

In [8]:
#setting variables for ideal heatmap
gplaces_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
gplaces_parameters = {
    'keyword' : 'hotel',
    'location' : '',
    'radius' : 5000,
    'key' : gkey
}

#setting lists/variables for forloop
found = False
hotel_names, hotel_lat, hotel_long = [], [], []
hotel_meta_list = [hotel_names, hotel_lat, hotel_long]
functions_list = ["hotel_response['results'][0]['name']", "hotel_response['results'][0]['geometry']['location']['lat']", "hotel_response['results'][0]['geometry']['location']['lng']"]

In [9]:
#requests hotel of ideal locations
for i in range(len(ideal_df['City'])):
    gplaces_parameters['location'] = f"{ideal_df['Latitude'].tolist()[i]},{ideal_df['Longitude'].tolist()[i]}"
    hotel_response = requests.get(gplaces_url, params = gplaces_parameters).json()
    for j in range(len(hotel_meta_list)):
        try:
            hotel_meta_list[j].append(eval(functions_list[j]))
            found = True
        except:
            hotel_meta_list[j].append(np.NaN)
    if (found == True):
        print(f"{ideal_df['City'].tolist()[i]} has at least one matching result.")
    else:
        print(f"{ideal_df['City'].tolist()[i]} has no matching results.")
    found = False #resetting for the next city

ginir has at least one matching result.
emerald has at least one matching result.
armilla has at least one matching result.
rawson has at least one matching result.


### Creating the new dataframe with hotel data

In [10]:
#variables for populating
new_columns = ['Closest Hotel', 'Hotel Latitude', 'Hotel Longitude']

#populating dataframe
for i in range(len(new_columns)):
    ideal_df[new_columns[i]] = hotel_meta_list[i]

#displaying new dataframe
ideal_df.head()

,City,Country,Latitude,Longitude,Temp (F),Humidity (%),Clouds (%),Wind Speed (mph),Closest Hotel,Hotel Latitude,Hotel Longitude
0,ginir,et,7.138672,40.711564,69.04,37.0,85.0,7.92,Bekera hotel,7.137535,40.712731
1,emerald,au,37.088557,-95.580680,76.98,33.0,92.0,3.49,Junction West Coffeyville,37.105189,-95.587528
2,armilla,es,37.151231,-3.616715,68.29,40.0,20.0,3.00,Sercotel Gran Hotel Luna de Granada,37.176552,-3.612778
3,rawson,ar,-43.299858,-65.099487,74.07,31.0,97.0,8.01,Hosteria Sampedro,-43.301509,-65.104922


In [11]:
hotel_info = []
for i in range(len(ideal_df['Country'])):
    hotel_info.append(f"Name\n{ideal_df['Closest Hotel'][i]}\nCity\n{ideal_df['City'][i]}\nCountry\n{ideal_df['Country'][i]}")
print(hotel_info)

['Name\nBekera hotel\nCity\nginir\nCountry\net', 'Name\nJunction West Coffeyville\nCity\nemerald\nCountry\nau', 'Name\nSercotel Gran Hotel Luna de Granada\nCity\narmilla\nCountry\nes', 'Name\nHosteria Sampedro\nCity\nrawson\nCountry\nar']


In [12]:
i = 0
print(f"Name\n{ideal_df['Closest Hotel'][i]}\nCity\n{ideal_df['City'][i]}\nCountry\n{ideal_df['Country'][i]}")

Name
Bekera hotel
City
ginir
Country
et


In [13]:
#generating markers variables
coordinates = list(zip(ideal_df['Latitude'].tolist(), ideal_df['Longitude'].tolist()))

#creating marker layer
marker_layer = gmaps.marker_layer(coordinates,
                                  info_box_content = hotel_info
                                 )

#adding markers to map
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…